In [1]:
using SparseArrays, LightGraphs
include("src/DC.jl")
using .DC

In [2]:
# define graph
L = 4;
G = grid([L,L], periodic = true)
Adj = adjacency_matrix(G);
N = L^2

#set couplings
J = Adj.* rand([-1,1],N,N);
#set fields
h = fill(0.1, N);

In [5]:
β = 1.0;
ψ = DC.IsingFG(N,β*h,β*J, :EnergyFun); # computes moments numerically by running the trace over {-1,1}^2
ψ = DC.IsingFG(N,β*h,β*J, :IsingPair); # computes moments analytically

#BP -> set η = 0.0 to compute BP fixed points (valid on any graph topology)
#ϕBP , res, it, epsv = density_consistency(ψ,N, epsconv = 1e-10, ρ = 0.9, closure = :DC, verbose = true, η = 0.);

In [6]:
ϕ , res, it, epsv = density_consistency(ψ,N; epsconv = 1e-5, ρ = 0.95, closure = :DC, verbose = true, Λ = 10., λ = 1e-8, η = 1 - 1e-2, convtype = :params);

Closure DC (η = 0.99) converged in 758 iterations w.r.t params - err params = 9.894054507242345e-6 - err moments = 2.0066885964009629e-7


In [7]:
ϕ.μt

16-element Array{Float64,1}:
  0.10644329067439319
 -0.18425866715769187
  0.18425866107277708
 -0.18425866069911653
  0.11787268093515968
  0.1842586684204847 
  0.1842586684204847 
  0.11828594568436733
 -0.18425865984354198
  0.18425866579281736
  0.18425866684877923
  0.10860424497968595
 -0.18425866804263827
 -0.18425866753735418
 -0.18425866323470358
  0.18425866323470358

In [8]:
ϕ.Σt

16×16 Array{Float64,2}:
  0.98867   -0.260564   0.0       …   0.0        0.0        0.0     
 -0.260564   0.966049  -0.966049      0.966049   0.0        0.0     
  0.0       -0.966049   0.966049      0.0       -0.966049   0.0     
 -0.297628   0.0       -0.966049      0.0        0.0       -0.966049
 -0.283817   0.0        0.0           0.0        0.0        0.0     
  0.0       -0.966049   0.0       …   0.0        0.0        0.0     
  0.0        0.0        0.966049      0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0          -0.966049   0.0        0.0     
  0.0        0.0        0.0       …   0.0       -0.966049   0.0     
  0.0        0.0        0.0           0.0        0.0        0.290405
 -0.28683    0.0        0.0           0.966049   0.0       -0.966049
  0.0        0.966049   0.0           0.966049   0.966049   0.0     
  0.0     

In [ ]:
ϕs = DCState(N,[length(ψ[a].idx) for a=1:length(ψ)])


In [ ]:
density_consistency(ψ,N; state  = ϕs,epsconv = 1e-5, ρ = 0.95, closure = :DC, verbose = true, Λ = 10., λ = 1e-8, η = 1.1, convtype = :moments);

In [ ]:
ϕs.Σt

In [ ]:
Float64(19)

In [ ]:
?DC.EnergyFun